In [1]:
import sys
import pathlib
#sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent)))
#sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().joinpath('Quantum_Annealing_for_Particle_Matching'))))
sys.path.append(str(pathlib.PurePath(pathlib.Path.cwd().parent)))
for p in sys.path:
    print(p)
import os
import time
import h5py
import cupy as np
#import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tqdm
import dwave
import dwave.system
import src
from src.particle_funcs import distance_matrix as distance_matrix
from src.particle_funcs import mixing_models, io
import src.leap_funcs.qubo.q_matrix as q_matrix
from src.leap_funcs import embedding_quality
from src.leap_funcs.qubo import filter_samples
from src._misc import compare_matrices

from src import h5py_funcs
from src.h5py_funcs import discoveries, init_custom_getstates, io, parameterstudy_using_info_file

c:\Users\adam-1aeqn8vhvpjnv4u\micromamba\envs\masterth_file_cupy\python312.zip
c:\Users\adam-1aeqn8vhvpjnv4u\micromamba\envs\masterth_file_cupy\DLLs
c:\Users\adam-1aeqn8vhvpjnv4u\micromamba\envs\masterth_file_cupy\Lib
c:\Users\adam-1aeqn8vhvpjnv4u\micromamba\envs\masterth_file_cupy

c:\Users\adam-1aeqn8vhvpjnv4u\micromamba\envs\masterth_file_cupy\Lib\site-packages
c:\Users\adam-1aeqn8vhvpjnv4u\micromamba\envs\masterth_file_cupy\Lib\site-packages\win32
c:\Users\adam-1aeqn8vhvpjnv4u\micromamba\envs\masterth_file_cupy\Lib\site-packages\win32\lib
c:\Users\adam-1aeqn8vhvpjnv4u\micromamba\envs\masterth_file_cupy\Lib\site-packages\Pythonwin
e:\Oliver\zz_temp\Quantum_Annealing_for_Particle_Matching
imported cupy
imported cupy
Custom getstate functions for dwave.cloud.config.models.ClientConfig, dwave.cloud.client.qpu.Client, dwave.cloud.solver.StructuredSolver, dwave.system.samplers.dwave_sampler.DWaveSampler, dwave.system.composites.embedding.FixedEmbeddingComposite have been initialized.


In [2]:
b = np.zeros((12000, 12000), dtype=np.float16)
print(b.nbytes/10e9)
print(b.dtype)
print((12000**2)*2*10e-9)

0.0288
float16
2.88


In [3]:
sim_path = pathlib.Path(r'E:\\Oliver\\zz_temp\\dem\\11_Particles_17332_CPU_16_GPU_0.rocky.files')
#sim_path = pathlib.Path(r'E:\Oliver\zz_temp\dem\\14_Particles_36308_CPU_16_GPU_0.rocky.files')
dem_data_dict = src.particle_funcs.io.read_dem_data(sim_path)

100%|██████████| 1201/1201 [00:12<00:00, 99.51it/s] 


In [4]:
print(dem_data_dict['rocky_simulation00000.rhs']['Particles']['particles_position']['data'].view((np.float64,3)).copy())
part_pos_array = np.array(dem_data_dict['rocky_simulation00000.rhs']['Particles']['particles_position']['data'].view((np.float64,3)))
part_pos_array = np.array(part_pos_array)
part_mat_id = np.zeros(part_pos_array.shape[0])
part_mat_id[np.where(part_pos_array[:,0]>0.0)] = 1
part_mat_id

[[-0.016825   -0.05        0.        ]
 [-0.02232426 -0.046825    0.        ]
 [-0.02232426 -0.053175    0.        ]
 ...
 [ 0.18601839  0.09562079 -0.008968  ]
 [ 0.18581863  0.09276814 -0.00329835]
 [ 0.18461491  0.09675742  0.00964453]]


array([0., 0., 0., ..., 1., 1., 1.])

In [5]:
file_list = list(dem_data_dict.keys())
file_list.remove('num_particles')
file_list.remove('particle_index')
num_timesteps = len(file_list)
num_particles = part_pos_array.shape[0]

In [6]:
datatype = np.float16
part_pos_array_all = np.zeros((num_timesteps,num_particles,3), dtype=datatype)
for i,f in enumerate(file_list):
    for j,a in enumerate(['x','y','z']):
            part_pos_array_all[i,:,j] = np.array(dem_data_dict[f]['Particles']['particles_position']['data'][a].astype(datatype))

In [7]:
mixing_models_names = ['average_height', 'nearest_neighbour', 'lacey_index', 'mixing_entropy', 'coordination_number_index', 'particle_scale_index']
mixing_models_dict = {id:[] for id in mixing_models_names}
mixing_models_dict

{'average_height': [],
 'nearest_neighbour': [],
 'lacey_index': [],
 'mixing_entropy': [],
 'coordination_number_index': [],
 'particle_scale_index': []}

In [8]:
print(part_pos_array_all.dtype)
distance_matrix.calc_phi_ij(part_pos_array_all[0],part_pos_array_all[0])

float16


array([[0.        , 0.00634296, 0.00634296, ..., 0.24991654, 0.24796877,
        0.24936118],
       [0.00634296, 0.        , 0.00634766, ..., 0.2525887 , 0.25063125,
        0.25203922],
       [0.00634296, 0.00634766, 0.        , ..., 0.25621741, 0.25422789,
        0.25570558],
       ...,
       [0.24991654, 0.2525887 , 0.25621741, ..., 0.        , 0.00635704,
        0.01870825],
       [0.24796877, 0.25063125, 0.25422789, ..., 0.00635704, 0.        ,
        0.0135866 ],
       [0.24936118, 0.25203922, 0.25570558, ..., 0.01870825, 0.0135866 ,
        0.        ]])

In [9]:
for t in tqdm.tqdm(range(num_timesteps)):
    mixing_models_dict['average_height'].append(mixing_models.average_height(coords=part_pos_array_all[t], axis=0, species=part_mat_id, normalize=0, verbose=0))
    #mixing_models_dict['nearest_neighbour'].append(mixing_models.nearest_neighbour(coords=part_pos_array_all[t], species=part_mat_id, num_neighbours=12, kwargs_distance_matrix={}, verbose=0))
    mixing_models_dict['lacey_index'].append(mixing_models.calc_lacey_index(coords=part_pos_array_all[t], species=part_mat_id, sample_method='random', sample_count=100, kwargs_distance_matrix={}, verbose=0))
    mixing_models_dict['mixing_entropy'].append(mixing_models.calc_mixing_entropy(coords=part_pos_array_all[t], species=part_mat_id, sample_method='random', sample_count=100, method='combined', kwargs_distance_matrix={}, verbose=0))
    mixing_models_dict['coordination_number_index'].append(mixing_models.calc_coordination_number_index(coords=part_pos_array_all[t], species=part_mat_id, diameter = 0.0, kwargs_distance_matrix={}, verbose=0, _internal=0))
    #mixing_models_dict['particle_scale_index'].append(mixing_models.calc_particle_scale_index(coords=part_pos_array_all[t], species=part_mat_id, diameter = 0.0, sample_method='random', sample_count=100, kwargs_distance_matrix={}, verbose=0))

100%|██████████| 1201/1201 [10:09:36<00:00, 30.46s/it] 


In [32]:
import pickle
import numpy

In [38]:
print(len(mixing_models_dict['mixing_entropy'][0]))

TypeError: len() of unsized object

In [50]:
mixing_models_dict_numpy = {}
for key, value in mixing_models_dict.items():
    print(key, value)
    if len(value)==0:
        continue
    if isinstance(value[0], list) and len(value[0])==1:
        print(value[0][0])
        mixing_models_dict_numpy[key] = numpy.array([v[0].get() for v in value])
    elif isinstance(value[0], list) and len(value[0])>1:
        raise ValueError(f'{key} value[0] is list of len>1, this is not handled yet')
    if isinstance(value[0], np.ndarray):
        mixing_models_dict_numpy[key] = numpy.array([v.get() for v in value])
mixing_models_dict_numpy

average_height [array([-0.06173706,  0.06173706]), array([-0.0604248 ,  0.06039429]), array([-0.06008911,  0.06011963]), array([-0.06005859,  0.06008911]), array([-0.06005859,  0.06008911]), array([-0.06008911,  0.06008911]), array([-0.06018066,  0.06015015]), array([-0.06030273,  0.06027222]), array([-0.06057739,  0.06048584]), array([-0.06100464,  0.06082153]), array([-0.06164551,  0.06140137]), array([-0.06262207,  0.06231689]), array([-0.06384277,  0.0635376 ]), array([-0.06542969,  0.06512451]), array([-0.06738281,  0.06695557]), array([-0.06964111,  0.06903076]), array([-0.07202148,  0.07147217]), array([-0.07489014,  0.07427979]), array([-0.07800293,  0.07727051]), array([-0.08129883,  0.08062744]), array([-0.08477783,  0.08404541]), array([-0.08837891,  0.08752441]), array([-0.09179688,  0.09082031]), array([-0.0947876 ,  0.09368896]), array([-0.09710693,  0.0960083 ]), array([-0.09875488,  0.09771729]), array([-0.09991455,  0.09893799]), array([-0.10076904,  0.09985352]), arra

{'average_height': array([[-0.06173706,  0.06173706],
        [-0.0604248 ,  0.06039429],
        [-0.06008911,  0.06011963],
        ...,
        [-0.05267334,  0.05310059],
        [-0.05163574,  0.05209351],
        [-0.05081177,  0.05130005]]),
 'lacey_index': array([[0.11164458, 0.11888349],
        [0.13479342, 0.12276492],
        [0.11062381, 0.11158508],
        ...,
        [0.57070964, 0.56536292],
        [0.54891726, 0.54724425],
        [0.58120588, 0.59237436]]),
 'mixing_entropy': array([3.88790875e-06, 1.04493469e-05, 1.58337749e-05, ...,
        1.70292696e-04, 1.31039326e-04, 1.30731303e-04]),
 'coordination_number_index': array([0.00147365, 0.0234248 , 0.02509368, ..., 0.50328781, 0.50314088,
        0.50212697])}

In [51]:
for key, value in mixing_models_dict_numpy.items():
    if len(value) == 0:
        continue
    print(key, type(value), type(value[0]))

average_height <class 'numpy.ndarray'> <class 'numpy.ndarray'>
lacey_index <class 'numpy.ndarray'> <class 'numpy.ndarray'>
mixing_entropy <class 'numpy.ndarray'> <class 'numpy.float64'>
coordination_number_index <class 'numpy.ndarray'> <class 'numpy.float64'>


In [52]:
with open(r'mixing_modles_dict_17k_parts.txt', 'wb') as f:
    pickle.dump(mixing_models_dict, f, protocol=4)
with open(r'mixing_modles_dict_numpy_17k_parts.txt', 'wb') as f:
    pickle.dump(mixing_models_dict_numpy, f, protocol=4)

In [ ]:
mixing_models.average_height(coords=part_pos_array_all[t], axis=0, species=part_mat_id, normalize=0, verbose=0)
mixing_models.nearest_neighbour(coords=part_pos_array_all[t], species=part_mat_id, num_neighbours=12, kwargs_distance_matrix={}, verbose=0)
mixing_models.calc_lacey_index(coords=part_pos_array_all[t], species=part_mat_id, sample_method='random', sample_count=100, kwargs_distance_matrix={}, verbose=0)
mixing_models.calc_mixing_entropy(coords=part_pos_array_all[t], species=part_mat_id, sample_method='random', sample_count=100, method='combined', kwargs_distance_matrix={}, verbose=0)
mixing_models.calc_coordination_number_index(coords=part_pos_array_all[t], species=part_mat_id, diameter = 0.0, kwargs_distance_matrix={}, verbose=0, _internal=0)
mixing_models.calc_particle_scale_index(coords=part_pos_array_all[t], species=part_mat_id, diameter = 0.0, sample_method='random', sample_count=100, kwargs_distance_matrix={}, verbose=0)